In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


# Caminho do arquivo
file_path = 'omni2_daily_19642024.lst'

# Ler o arquivo com as colunas especificadas
data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=[
    'Year', 'DOY', 'Hour', 'f10.7_index'])

# Remover linhas com valores NaN (se houver)
data.dropna(inplace=True)

# Converter o tempo em uma escala contínua de anos
data['Time'] = data['Year'] + (data['DOY'] - 1) / 365

data_filtered = data[['Time', 'f10.7_index']]

# Exibir as primeiras linhas para verificar
print(data_filtered.head())

# Salvar o dataset limpo (opcional)
data_filtered.to_csv('omni2_cleaned.csv', index=False)

# Verificar o tamanho do dataset após a limpeza
print(f"Total de linhas após a limpeza: {len(data_filtered)}")


          Time  f10.7_index
0  1964.000000        999.9
1  1964.002740         68.3
2  1964.005479         70.7
3  1964.008219         70.4
4  1964.010959         71.3
Total de linhas após a limpeza: 21925


<ipython-input-2-89a727cfa5c2>:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=[


In [3]:
# Estatísticas descritivas
print(data_filtered.describe())

# Verificar valores nulos
print(data_filtered.isnull().sum())

# Correlação entre variáveis
print(data_filtered.corr())


               Time   f10.7_index
count  21925.000000  21925.000000
mean    1994.011636    118.688292
std       17.328801     54.216421
min     1964.000000     63.400000
25%     1979.005479     76.200000
50%     1994.010959    103.900000
75%     2009.016438    148.000000
max     2024.024658    999.900000
Time           0
f10.7_index    0
dtype: int64
                 Time  f10.7_index
Time         1.000000    -0.135105
f10.7_index -0.135105     1.000000


In [4]:
# Separar variáveis independentes e dependentes
X = data_filtered[['Time']]
y = data_filtered['f10.7_index']

# Dividir em conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Modelo 1: Árvore de Decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)

In [6]:
mse_tree

974.0234007981755

In [7]:
# Modelo 2: Regressão Linear
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
mse_linear


3476.8464403460616

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Usar 'Time' como índice para séries temporais
data_filtered.set_index('Time', inplace=True)

# Agrupar por média mensal para reduzir o volume de dados (opcional)
data_filtered = data_filtered.resample('M').mean()

# Dividir os dados em treino (80%) e teste (20%)
train_size = int(0.8 * len(data_filtered))
train_sarima = data_filtered['f10.7_index'][:train_size]
test_sarima = data_filtered['f10.7_index'][train_size:]

# Ajustar modelo SARIMA simplificado (p=0, d=1, q=1) com sazonalidade reduzida (P=0, D=1, Q=1, m=12)
sarima_model = SARIMAX(
    train_sarima,
    order=(0, 1, 1),
    seasonal_order=(0, 1, 1, 12),  # Usando sazonalidade mensal
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarima_model_fit = sarima_model.fit(disp=False)

# Previsão usando predict(), especificando o intervalo
start_index = test_sarima.index[0]
end_index = test_sarima.index[-1]
y_pred_sarima = sarima_model_fit.predict(start=start_index, end=end_index)

# Calcular o Erro Quadrático Médio (MSE)
mse_sarima = mean_squared_error(test_sarima, y_pred_sarima)
print(f"MSE do SARIMA otimizado: {mse_sarima:.4f}")

# Visualizar os resultados
plt.figure(figsize=(12, 6))
plt.plot(train_sarima.index, train_sarima, label='Treino', color='blue')
plt.plot(test_sarima.index, test_sarima, label='Teste Real', color='black')
plt.plot(y_pred_sarima.index, y_pred_sarima, label='Previsão SARIMA', color='red', linestyle='--')
plt.xlabel('Ano')
plt.ylabel('Índice F10.7')
plt.title('Previsão com SARIMA Otimizado')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'data_filtered' is not defined

In [ ]:
# =======================
# Comparação dos Resultados
# =======================
print(f"Erro Quadrático Médio (MSE) para cada modelo:")
print(f"Árvore de Decisão: {mse_tree:.4f}")
print(f"Regressão Linear: {mse_linear:.4f}")
print(f"ARIMA: {mse_sarima:.4f}")

# =======================
# Visualização dos Resultados
# =======================
plt.figure(figsize=(12, 6))
plt.plot(X_test['Time'], y_test, label='Dados Reais', color='black')
plt.scatter(X_test['Time'], y_pred_tree, label='Previsão Árvore de Decisão', color='blue', alpha=0.5)
plt.plot(X_test['Time'], y_pred_linear, label='Previsão Regressão Linear', color='green', linestyle='--')
plt.plot(test_arima.index, y_pred_arima, label='Previsão ARIMA', color='red')
plt.xlabel('Ano')
plt.ylabel('Índice F10.7')
plt.title('Comparação de Modelos de Previsão')
plt.legend()
plt.grid(True)
plt.show()